# Primeiro contato com a biblioteca PDAL

Esse primeiro notebook tem como objetivo introduzir ao acesso da base de dados EntiWine atravéz da biblioteca PDAL

In [1]:
import pdal
import json

## Acessando os dados

Primeiro é necessário acessar a nuvem de pontos. Podemos fazer isso por um serviço EPT (EntWine Tile of Points). Nesse caso vamos escolher uma área pequena, do coreto do Parque da Luz.

In [2]:
bounds = ([333060, 333090], [7396500, 7396530])

In [3]:
str(bounds)

'([333060, 333090], [7396500, 7396530])'

In [4]:
ept = [
    {
      "type": "readers.ept",
      "filename": "https://ept-m3dc-pmsp.s3-sa-east-1.amazonaws.com/ept.json",
      "bounds": str(bounds)
    },
    {   
        "type":"filters.hag_delaunay"
#     },
#     {   "type":"filters.eigenvalues",
#         "knn":16
    },
#     {   "type":"filters.normal",
#         "knn":16
#     },
#     {
#         "type":"filters.cluster"
#     },
]

pipeline = pdal.Pipeline(json.dumps(ept))
pipeline.validate()
%time n_points = pipeline.execute()
print(f'Pipeline selected {n_points} points')

CPU times: user 1.44 s, sys: 47.6 ms, total: 1.49 s
Wall time: 2.26 s
Pipeline selected 13496 points


## Estrutura de dados

Os dados estão estruturados e podemos usar a biblioteca Pandas para acessá-los

In [5]:
pipeline.schema

{'schema': {'dimensions': [{'name': 'X', 'size': 8, 'type': 'floating'},
   {'name': 'Y', 'size': 8, 'type': 'floating'},
   {'name': 'Z', 'size': 8, 'type': 'floating'},
   {'name': 'Intensity', 'size': 2, 'type': 'unsigned'},
   {'name': 'ReturnNumber', 'size': 1, 'type': 'unsigned'},
   {'name': 'NumberOfReturns', 'size': 1, 'type': 'unsigned'},
   {'name': 'ScanDirectionFlag', 'size': 1, 'type': 'unsigned'},
   {'name': 'EdgeOfFlightLine', 'size': 1, 'type': 'unsigned'},
   {'name': 'Classification', 'size': 1, 'type': 'unsigned'},
   {'name': 'ScanAngleRank', 'size': 4, 'type': 'floating'},
   {'name': 'UserData', 'size': 1, 'type': 'unsigned'},
   {'name': 'PointSourceId', 'size': 2, 'type': 'unsigned'},
   {'name': 'GpsTime', 'size': 8, 'type': 'floating'},
   {'name': 'Red', 'size': 2, 'type': 'unsigned'},
   {'name': 'Green', 'size': 2, 'type': 'unsigned'},
   {'name': 'Blue', 'size': 2, 'type': 'unsigned'},
   {'name': 'OriginId', 'size': 4, 'type': 'unsigned'},
   {'name': '

In [6]:
import pandas as pd
arr = pipeline.arrays[0]
df = pd.DataFrame(arr)
# print(df.head().to_latex(index=False))
df.head()

,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,UserData,PointSourceId,GpsTime,Red,Green,Blue,OriginId,HeightAboveGround
0,333069.84,7396523.41,737.12,6,2,2,1,0,2,18.0,0,6,358517.058422,22272,24320,23808,2416,0.000000
1,333060.54,7396515.26,737.35,14,1,1,1,0,2,-10.0,0,6,358026.122710,7168,10240,15616,2416,0.000000
2,333080.45,7396516.96,737.70,9,2,2,0,0,2,-6.0,0,6,358350.736322,36864,28672,26624,2416,0.000000
3,333084.40,7396519.07,752.30,30,1,1,1,0,5,-10.0,15,6,358025.631887,17920,23552,16896,2416,14.927621
4,333082.83,7396519.22,746.82,14,2,2,1,0,5,-10.0,9,6,358025.668248,16384,21504,15360,2416,9.487211


In [7]:
df.dtypes

X                    float64
Y                    float64
Z                    float64
Intensity             uint16
ReturnNumber           uint8
NumberOfReturns        uint8
ScanDirectionFlag      uint8
EdgeOfFlightLine       uint8
Classification         uint8
ScanAngleRank        float32
UserData               uint8
PointSourceId         uint16
GpsTime              float64
Red                   uint16
Green                 uint16
Blue                  uint16
OriginId              uint32
HeightAboveGround    float64
dtype: object

## Classificação

Os dados já foram previamente classificados conforme classes da ASPRS

In [8]:
df.Classification.unique()

array([ 2,  5,  6, 19, 20], dtype=uint8)

## Normalizando coordenadas para visualização


In [9]:
# Reduzindo valores a mínimos para poderem ser visualizados
df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']] - df[['X', 'Y', 'Z']].min()

In [10]:
import ipyvolume as p3 

fig = p3.figure(width=1000)
fig.xlabel='Y'
fig.ylabel='Z'
fig.zlabel='X'
all_points = p3.scatter(df['Y'], df['Z'], df['X'], color='grey', size=1, marker='square_2d')
# ground = p3.scatter(gd['Y'], gd['Z'], gd['X'], color='grey', size=.2)
# building = p3.scatter(bd['Y'], bd['Z'], bd['X'], color='orange', size=.3)
# vegetation = p3.scatter(vg['Y'], vg['Z'], vg['X'], color='green', size=.1)
p3.squarelim()
p3.show()